In [167]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
from tqdm import tqdm


%matplotlib inline

In [6]:
file_path = './Data/list_attr_celeba.txt'
file_content = ''

with open(file_path, "r") as file:
    # Read the entire file content into a single string
    file_content = file.read()
    file_content = file_content.replace('  ', ',')
    file_content = file_content.replace(' -', ',-')
    file_content = file_content.replace(' ', ',')


file_path = "attributes.csv"

with open(file_path, "w") as file:
    file.write(file_content)

In [210]:
df = pd.read_csv('./Data/attributes.csv')

In [211]:
df = df.replace(-1, 0)
df = df.iloc[14338:]

In [212]:
train_df, test_df = train_test_split(df, train_size=1000, random_state=42)

In [213]:
train_df

,Image,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
25292,025293.jpg,0,0,1,0,0,0,0,0,1,...,0,1,0,1,1,0,1,0,0,1
15184,015185.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
67779,067780.jpg,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
27572,027573.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
126559,126560.jpg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134217,134218.jpg,0,1,1,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1
118032,118033.jpg,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1
146270,146271.jpg,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
161205,161206.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [221]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, dataframe):
        self.data_dir = data_dir
        self.dataframe = dataframe
        self.image_paths = self.dataframe['Image'].tolist()
        self.labels = self.dataframe.drop('Image', axis=1).values.astype(np.float32)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.image_paths[idx])
        image = Image.open(img_name)
        transform = transforms.Compose([transforms.ToTensor()])
        
        image = transform(image)

        label = torch.tensor(self.labels[idx])
        return image, label

In [222]:
train_dataset = CustomImageDataset('./Data/img_align_celeba/', train_df)

In [223]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [224]:
resnet50 = models.resnet50(pretrained=True) # load in the pre trained resnet18 model

/Users/samvasserman/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/samvasserman/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [225]:
num_labels_train = len(train_df.columns) - 1
num_labels_test = len(test_df.columns) - 1
print(num_labels_train, num_labels_test)

40 40


In [231]:
in_features = resnet50.fc.in_features

num_attributes = 40  # Number of attributes

resnet50.fc = nn.Sequential(
    nn.Linear(in_features, num_attributes),
    nn.Sigmoid()
)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=1)

AttributeError: 'Sequential' object has no attribute 'in_features'

In [232]:
num_epochs = 5  # Adjust as needed

for epoch in range(num_epochs):
    resnet50.train()
    runningLoss = 0
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()

    
    # Print statistics or log them as needed
    print(f'Epoch {epoch+1}/{num_epochs}, Running Loss: {runningLoss}')


100%|███████████████████████████████████████████| 32/32 [04:21<00:00,  8.18s/it]


Epoch 1/5, Running Loss: 20.917702674865723


100%|███████████████████████████████████████████| 32/32 [04:16<00:00,  8.00s/it]


Epoch 2/5, Running Loss: 20.852234601974487


 16%|██████▉                                     | 5/32 [00:50<04:33, 10.14s/it]


KeyboardInterrupt: 

In [229]:
torch.save(resnet50, 'model_good.pth')